In [1]:
'''
导入所有需要的库
'''
import requests
import xlwt
import xlrd
import csv
import pprint
import time
from IPython import embed
from tqdm import tqdm
'''
建立空xls文档
'''
wb = xlwt.Workbook()
ws = wb.add_sheet('深圳',cell_overwrite_ok=True)

In [2]:
from geopy.distance import geodesic

In [3]:
import numpy as np

In [4]:
'''
This function is used to get all information we need using "keyword search" function in Gaode platform. 
'''
def poi_search(key,locations,types,radius):
    import json
    url_0 = 'https://restapi.amap.com/v3/place/around?'
    key = str(key)
    types = str(types)
    location = str(locations)
    radius = str(radius)
    extensions = 'all'

    url_lng_lat = url_0 + 'key=' + key + '&location=' + locations + '&types=' + types  + '&radius=' + radius + '&extensions=' + extensions
    data = []
    for p in range(1,50):
        url = url_lng_lat + '&page=' + str(p)
        html = requests.get(url)
        if html.status_code == 200:
            
            content = json.loads(html.text)
            dic = content["pois"]
            for i in dic:
                location = i["location"]
                name = i["name"]
                datas = [name,location]
                data.append(datas)

    return data

In [5]:
'''
Personalize your search.
'''
types = str(input("请输入你需要查询的公共服务设施类型：\n"))
weight = str(input("请输入权重：\n"))

请输入你需要查询的公共服务设施类型：
150500|150600|150700
请输入权重：
0.75


In [6]:
'''
This function is used to get all information we need using "type search" function in Gaode platform. 
'''
def poi_search_keywords(key,locations,keywords,radius):
    import json
    url_0 = 'https://restapi.amap.com/v3/place/around?'
    key = str(key)
    keywords = str(keywords)
    location = str(locations)
    radius = str(radius)
    extensions = 'all'

    url_lng_lat = url_0 + 'key=' + key + '&location=' + locations + '&keywords=' + keywords + '&radius=' + radius + '&extensions=' + extensions
    data = []
    for p in range(1,50):
        url = url_lng_lat + '&page=' + str(p)
        html = requests.get(url)
        if html.status_code == 200:
            
            content = json.loads(html.text)
            dic = content["pois"]
            for i in dic:
                location = i["location"]
                name = i["name"]
                datas = [name,location]
                data.append(datas)

    return data

In [ ]:
'''
Personalize your search.
'''
keywords = str(input("请输入你需要查询的公共服务设施类型：\n"))
weight = str(input("请输入权重：\n"))

In [6]:
'''
This function is used to get the shortest path 
and its correspond walk time between point "origin" and point "destination".
'''
def accessibility(key,origin,destination):
    import json
    url = "https://restapi.amap.com/v3/direction/walking?parameters"
    keyword = {"origin": origin,"destination": destination,"key": key,"output":"JSON"}
    access = []
    b = requests.get(url, params=keyword)
    if b.status_code == 200:
        html = b.text
        json = json.loads(html)
        route = json["route"]["paths"][0]
        distance = route["distance"]
        duration = route["duration"]
        access.append(distance)
        access.append(duration)
    return access


In [7]:
'''
Change the sequence of latitude and longitude of the information we get before.
'''
def transposexy(x):
    x_spit = x.split(",")
    x_spitnp = np.array(x_spit)
    #print(x_spitnp)
    x_spitnp[[0,1]] = x_spitnp[[1,0]]
    #print(x_spitnp)
    x_spitnp=','.join(x_spitnp)
    return x_spitnp

In [8]:
'''
save information into csv format
'''
def csv_save(row):
    with open("data_address_gz_0.75.csv", 'a') as f:
        write = csv.writer(f)
        write.writerow(row)
'''
find POIs based on community addressses in this txt
'''
def address_get():
    f = open(r'address_gz.txt', 'r')
    a = list(f)
    b = []
    for i in a:
        a = i.replace("  ", ",").replace("\n", "")
        b.append(a)
    return b

分类爬取

In [9]:
'''
input information we need
'''
print("【基础数据】")
key = str(input("请输入你的高德API密钥：\n"))
'''key = '7f4837aea08366cc4fd9a83c6b340bf7'

key = "a1cf852d9781d1e427a06ab57400ce13"

key = "153c2c2f24af71aeab07f04018732476"
'''
radius = int(input("请输入你需要查询的范围（单位：m）：\n"))
datas = []

address = address_get()
title = ["步行最短距离（m）","步行最短时间（s）","公共服务设施名称","ID",'距离分类','权重','纠正后权重']
datas.append(title)

【基础数据】
请输入你的高德API密钥：
a1cf852d9781d1e427a06ab57400ce13
请输入你需要查询的范围（单位：m）：
1200


In [10]:
'''
start getting information
'''
print("\n【数据采集】")
print("数据采集进度：")
number = 0
for x in tqdm(address):
    #print(x)
    #print(type(x))
    number = number + 1
    a = poi_search(key, x,types,radius)#search using keywords
    b = poi_search_keywords(key,x,keywords,radius)#search using types
    
    #print(a)
    for i in a:
        #a = accessibility(key, x, loca,headers)
        #print(a[0])
        #if int(a[0]) > 1305:
            #continue
        name = i[0]
        loca = i[1]
        #print(loca)
        #print(type(loca))
        #dist = geodesic((transposexy(x)), (transposexy(loca))).m
        a = accessibility(key, x, loca)
        print(a[0])
        if int(a[0]) > 1305:#select all POIs we need 
            continue
        a.append(name)
        a.append(str(number))
        #a.append(str(dist))
        a0 = int(a[0])#decay function based on distance
        if a0 < 435:
            ind = 1
        elif a0 < 652.5 and a0 >= 435:
            ind = 0.9
        elif a0 < 870 and a0 >= 652.5:
            ind = 0.55
        elif a0 < 1087.5 and a0 >= 870:
            ind = 0.25
        elif a0 < 1305 and a0 >= 1087.5:
            ind = 0.08
            
        a.append(str(ind))
        a.append(str(weight))
        a.append(str(round((float(weight)*ind),4)))#calculate new weight based on the decay function above
        datas.append(a)
    
    for j in b:
        name = j[0]
        loca = j[1]
        a = accessibility(key, x, loca)
        print(a[0])
        if int(a[0]) > 1305:
            continue
        a.append(name)
        a.append(str(number))
        #a.append(str(dist))
        a0 = int(a[0])#same decay function based on distance
        if a0 < 435:
            ind = 1
        elif a0 < 652.5 and a0 >= 435:
            ind = 0.9
        elif a0 < 870 and a0 >= 652.5:
            ind = 0.55
        elif a0 < 1087.5 and a0 >= 870:
            ind = 0.25
        elif a0 < 1305 and a0 >= 1087.5:
            ind = 0.08
            
        a.append(str(ind))
        a.append(str(weight))
        a.append(str(round((float(weight)*ind),4)))#calculate new weight based on the decay function above
        datas.append(a)
        

  0%|          | 0/171 [00:00<?, ?it/s]


【数据采集】
数据采集进度：
340
429
450
511
728
805
438
711
563
678
603
664
709
788
639
1621
817
833
907
964
982
1076
981
924
1561
1001
1968
1015
1034
1125
1047
1217
899
1591
1215
1322
1099
1325
1691
1438
1397
1037
1054
1356
1478
1451
1489
1386
1345
1359
1496
1593
1566
1361
1597
1473
1431
1365
1514
1679
1520


  1%|          | 1/171 [00:22<1:04:04, 22.61s/it]

1605
1462
429
429
469
433
3687
1429
766
1019
1450
2043


  1%|          | 2/171 [00:34<45:28, 16.15s/it]  

...

1602
1559
73
113
253
414
316
521
498
601
548
661
948
1141
883
1002
1121
1066
1003
1258
1228
1474
1459
1386


 99%|█████████▉| 170/171 [27:59<00:12, 12.77s/it]

1464
433
666


100%|██████████| 171/171 [28:09<00:00,  9.88s/it]

1269


In [11]:
for i in datas:
    csv_save(i)
print("\n-------------------数据采集完成-------------------")
pprint.pprint(datas)


-------------------数据采集完成-------------------
[['步行最短距离（m）', '步行最短时间（s）', '公共服务设施名称', 'ID', '距离分类', '权重', '纠正后权重'],
 ['340', '272', '文德南站(公交站)', '1', '1', '0.75', '0.75'],
 ['429', '343', '东堤站(公交站)', '1', '1', '0.75', '0.75'],
 ['450', '360', '珠光路总站(公交站)', '1', '0.9', '0.75', '0.675'],
 ['511', '409', '万福路站(公交站)', '1', '0.9', '0.75', '0.675'],
 ['728', '582', '团一大广场(地铁站)', '1', '0.55', '0.75', '0.4125'],
 ['805', '644', '团一大广场(公交站)', '1', '0.55', '0.75', '0.4125'],
 ['438', '350', '万福东站(公交站)', '1', '0.9', '0.75', '0.675'],
 ['711', '569', '越秀南路站(公交站)', '1', '0.55', '0.75', '0.4125'],
 ['563', '450', '天字码头站(公交站)', '1', '0.9', '0.75', '0.675'],
 ['678', '542', '万福西(公交站)', '1', '0.55', '0.75', '0.4125'],
 ['603', '482', '珠光路(公交站)', '1', '0.9', '0.75', '0.675'],
 ['664', '531', '南关(公交站)', '1', '0.55', '0.75', '0.4125'],
 ['709', '567', '文德路总站(公交站)', '1', '0.55', '0.75', '0.4125'],
 ['788', '630', '北京路(地铁站)', '1', '0.55', '0.75', '0.4125'],
 ['639', '511', '中山图书馆站(公交站)', '1', '0.9', '0.75',

 ['287', '230', '黄边南街(公交站)', '70', '1', '0.75', '0.75'],
 ['282', '226', '乐得花园站(公交站)', '70', '1', '0.75', '0.75'],
 ['435', '348', '黄边北路西(公交站)', '70', '0.9', '0.75', '0.675'],
 ['458', '366', '黄边横路站(公交站)', '70', '0.9', '0.75', '0.675'],
 ['529', '423', '黄边横路(临时站)(公交站)', '70', '0.9', '0.75', '0.675'],
 ['792', '634', '尖彭路口(公交站)', '70', '0.55', '0.75', '0.4125'],
 ['743', '594', '鹤边员村站(公交站)', '70', '0.55', '0.75', '0.4125'],
 ['833', '666', '省戒毒基地(公交站)', '70', '0.55', '0.75', '0.4125'],
 ['803', '642', '黄边北路中站(公交站)', '70', '0.55', '0.75', '0.4125'],
 ['808', '646', '黄边北路中(临时站)(公交站)', '70', '0.55', '0.75', '0.4125'],
 ['1118', '894', '司法学校(公交站)', '70', '0.08', '0.75', '0.06'],
 ['918', '734', '嘉禾(公交站)', '70', '0.25', '0.75', '0.1875'],
 ['1117', '894', '百花岭北街口站(公交站)', '70', '0.08', '0.75', '0.06'],
 ['1198', '958', '时代玫瑰园(公交站)', '70', '0.08', '0.75', '0.06'],
 ['1287', '1030', '鹤边军民西路站(公交站)', '70', '0.08', '0.75', '0.06'],
 ['595', '476', '鹤南(地铁站)', '70', '0.9', '0.75', '0.675'],
 ['375',

 ['652', '522', '龙口西站(公交站)', '116', '0.9', '0.75', '0.675'],
 ['698', '558', '天河北公交总站(公交站)', '116', '0.55', '0.75', '0.4125'],
 ['632', '506', '龙口西路北(公交站)', '116', '0.9', '0.75', '0.675'],
 ['619', '495', '天河北路站(公交站)', '116', '0.9', '0.75', '0.675'],
 ['744', '595', '天润路口(公交站)', '116', '0.55', '0.75', '0.4125'],
 ['1137', '910', '龙口西路南(公交站)', '116', '0.08', '0.75', '0.06'],
 ['588', '470', '天河东路口(公交站)', '116', '0.9', '0.75', '0.675'],
 ['868', '694', '穗园小区总站(公交站)', '116', '0.55', '0.75', '0.4125'],
 ['777', '622', '林乐路站(公交站)', '116', '0.55', '0.75', '0.4125'],
 ['928', '742', '穗园小区(公交站)', '116', '0.25', '0.75', '0.1875'],
 ['922', '738', '龙口东站(公交站)', '116', '0.25', '0.75', '0.1875'],
 ['756', '605', '天寿路站(公交站)', '116', '0.55', '0.75', '0.4125'],
 ['974', '779', '龙口西总站(公交站)', '116', '0.25', '0.75', '0.1875'],
 ['1111', '889', '龙口东路北(公交站)', '116', '0.08', '0.75', '0.06'],
 ['1131', '905', '岗顶(快速公交站)', '116', '0.08', '0.75', '0.06'],
 ['1243', '994', '体育中心东门(公交站)', '116', '0.08', '0.75', 

1. 断线再运行可以继续写


开始打分